In [7]:
from numba import jit, cuda 
import numpy as np 
# to measure exec time 
from timeit import default_timer as timer    

In [8]:
import numpy as np
from pyculib import rand as curand

prng = curand.PRNG(rndtype=curand.PRNG.XORWOW)
rand = np.empty(100000)
prng.uniform(rand)
print rand[:10]

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(rand[:10])? (<ipython-input-8-50f5bb65583a>, line 7)

In [9]:
# normal function to run on cpu 
def func(a):                                 
    for i in range(10000000): 
        a[i]+= 1

In [10]:
# function optimized to run on gpu  
@jit(target ="cuda")                          
def func2(a): 
    for i in range(10000000): 
        a[i]+= 1

In [11]:
if __name__=="__main__": 
    n = 10000000                            
    a = np.ones(n, dtype = np.float64) 
    b = np.ones(n, dtype = np.float32) 
      
    start = timer() 
    func(a) 
    print("without GPU:", timer()-start)     
      
    start = timer() 
    func2(a)
    print("with GPU:", timer()-start) 

without GPU: 3.7972485600002983


/home/david/anaconda3/lib/python3.7/site-packages/numba/cuda/decorators.py:116: UserWarning: autojit is deprecated and will be removed in a future release. Use jit instead.
  warn('autojit is deprecated and will be removed in a future release. Use jit instead.')


NotImplementedError: bounds checking is not supported for CUDA

In [15]:
import jax.numpy as np
from jax import jit

def slow_f(x):
  # Element-wise ops see a large benefit from fusion
  return x * x + x * 2.0

x = np.ones((5000, 5000))
fast_f = jit(slow_f)
%timeit -n10 -r3 fast_f(x)  # ~ 4.5 ms / loop on Titan X
%timeit -n10 -r3 slow_f(x)  # ~ 14.5 ms / loop (also on GPU via JAX)

The slowest run took 9.56 times longer than the fastest. This could mean that an intermediate result is being cached.
11 ms ± 8.18 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)
66.9 ms ± 2.94 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [16]:
from jax import grad, jit
import jax.numpy as np

def sigmoid(x):
    return 0.5 * (np.tanh(x / 2.) + 1)

# Outputs probability of a label being true according to logistic model.
def logistic_predictions(weights, inputs):
    return sigmoid(np.dot(inputs, weights))

# Training loss is the negative log-likelihood of the training labels.
def loss(weights, inputs, targets):
    preds = logistic_predictions(weights, inputs)
    label_logprobs = np.log(preds) * targets + np.log(1 - preds) * (1 - targets)
    return -np.sum(label_logprobs)

# Build a toy dataset.
inputs = np.array([[0.52, 1.12,  0.77],
                   [0.88, -1.08, 0.15],
                   [0.52, 0.06, -1.30],
                   [0.74, -2.49, 1.39]])
targets = np.array([True, True, False, True])

# Define a compiled function that returns gradients of the training loss
training_gradient_fun = jit(grad(loss))

# Optimize weights using gradient descent.
weights = np.array([0.0, 0.0, 0.0])
print("Initial loss: {:0.2f}".format(loss(weights, inputs, targets)))
for i in range(100):
    weights -= 0.1 * training_gradient_fun(weights, inputs, targets)

print("Trained loss: {:0.2f}".format(loss(weights, inputs, targets)))

Initial loss: 2.77
Trained loss: 0.17
